In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10353636741638184
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 14.506126708482794
200000 26.903946346579996
300000 29.717101732276646
400000 29.257729068848672
500000 29.072114128241527
600000 30.019759848159175
700000 29.377533541693428
800000 29.013051896895497
900000 28.990088057346508
1000000 29.021389998173333
1100000 28.50015091427595
1200000 12.448604909848514
1300000 24.39422569431696
1400000 28.676666226766052
1500000 28.708725571902693
1600000 28.652787972599786
1700000 28.549707752209102
1800000 30.478627827766818
1900000 28.933127729496565
2000000 28.62672163660124
2100000 28.91091966005461
2200000 28.78523840724481
2300000 29.33414500849689
2400000 7.632688303849615
2500000 30.648942734342196
2600000 36.323631610429466
2700000 35.448663730386315
2800000 35.366707295553255
2900000 35.211725152059145
300000

29400000 19.460993663285254
29500000 16.360031383703678
29600000 27.094621095072895
29700000 27.703601036757814
29800000 27.845256206987315
29900000 28.666939037800663
30000000 27.409169652682383
30100000 29.08824095444781
30200000 28.924689837413762
30300000 28.996909587444076
30400000 12.205790195322258
30500000 28.941551212870824
30600000 28.684831217802103
30700000 30.156893034071626
30800000 25.44783810660691
30900000 14.89918116135068
31000000 28.806295993951924
31100000 29.054673456672887
31200000 28.666825733956255
31300000 27.94657496876249
31400000 27.697455189135386
31500000 27.713645682950233
31600000 27.175030731138456
31700000 25.949783838483388
31800000 7.767164645794505
31900000 27.029410548928613
32000000 25.893896900697392
32100000 27.055755515938507
32200000 27.586731015781165
32300000 28.7341438215273
32400000 27.9000509143856
32500000 29.15181880074732
32600000 28.419240699441993
32700000 19.251636443797445
32800000 21.339949171466227
32900000 28.898473760125224
33

59200000 27.0786555306113
59300000 11.455341719641977
59400000 24.211505959865555
59500000 25.29112145538919
59600000 25.01618279708908
59700000 23.494718347419287
59800000 23.417101732674563
59900000 23.35487458579073
60000000 13.494891993842428
60100000 17.251642819252112
60200000 23.85237830120927
60300000 23.279201131241127
60400000 23.60787673271066
60500000 24.494081317225643
60600000 24.37915622470149
60700000 21.001075144500703
60800000 16.16120429150831
60900000 26.522786852328405
61000000 26.670723349488657
61100000 26.04542076573402
61200000 26.554748242537364
61300000 26.555255739605258
61400000 26.507443278413763
61500000 27.04693314310307
61600000 25.758213920799793
61700000 10.152659884831008
61800000 25.620061485939786
61900000 24.92509837314285
62000000 24.007872227276685
62100000 24.284792728873477
62200000 23.47167597033074
62300000 24.231539181615275
62400000 5.936511868029489
62500000 24.598989653578858
62600000 23.260508415514323
62700000 23.444083383652483
628000

88900000 21.825156894043978
89000000 22.361027912913965
89100000 22.532746790842587
89200000 23.67429965240704
89300000 22.263323745160225
89400000 23.86818145345638
89500000 23.98406147176073
89600000 23.78426382925577
89700000 24.19858614932202
89800000 23.473982453555347
89900000 23.235963212260163
90000000 22.269198951901533
90100000 12.101860244686657
90200000 17.796466521367815
90300000 20.949760960959157
90400000 22.3326265766367
90500000 21.409564937553398
90600000 9.490229504970412
90700000 19.378475172092365
90800000 21.608703290813523
90900000 20.715106210637057
91000000 21.18164363149988
91100000 10.662800621760306
91200000 19.65149567444427
91300000 22.529700534150127
91400000 22.392953580332062
91500000 22.897914607315027
91600000 23.00863505128788
91700000 23.520561415824076
91800000 23.995053635858298
91900000 23.090163453454608
92000000 23.217333625810458
92100000 23.722362377219604
92200000 23.0149886591696
92300000 22.239881923692664
92400000 21.87322818378192
925000

118000000 20.705130535108534
118100000 19.607804405784872
118200000 19.846113827292488
118300000 19.066714187617656
118400000 19.364432293357755
118500000 16.822415378587454
118600000 9.462148834654045
118700000 20.06781467692031
118800000 13.299907227289609
118900000 11.14490710643047
119000000 19.345043700570677
119100000 12.12369918817296
119200000 15.45700544553708
119300000 19.251225219909283
119400000 18.87927838946506
119500000 19.501401903646464
119600000 19.65814707855266
119700000 20.214824042772904
119800000 20.517953042178565
119900000 20.993263893258188
120000000 20.63687939825555
120100000 21.404237679243316
120200000 20.78512930887554
120300000 20.330296259092858
120400000 19.287783949138955
120500000 19.52441827910207
120600000 19.724641047772153
120700000 18.97407415656443
120800000 19.13285567775877
120900000 12.198964718913771
121000000 13.57313831201497
121100000 18.039789274903633
121200000 5.499908801745423
121300000 18.60716494259329
121400000 16.533120824465296


...

261200000 10.411657653978784
261300000 9.928212766506785
261400000 10.1601799172233
261500000 8.530113483890739
261600000 5.844220768275495
261700000 9.942528303391853
261800000 9.466798854747106
261900000 9.45109775292499
262000000 9.409991785784174
262100000 9.01384111143412
262200000 9.02036750580281
262300000 7.301446964763744
262400000 4.82968280458366
262500000 8.821205395748406
262600000 9.008162315589805
262700000 9.211216235110038
262800000 9.458683679730608
262900000 9.4582649381658
263000000 9.961566939503298
263100000 8.101465296166369
263200000 5.787357412990362
263300000 10.06722578183513
263400000 10.051928791064533
263500000 10.248003672981438
263600000 10.101027293945293
263700000 10.016109674935395
263800000 9.96457521139658
263900000 3.891766858115502
264000000 10.763761987025433
264100000 9.917638141615493
264200000 9.515013229523829
264300000 9.541660863430769
264400000 9.506405412571585
264500000 9.132127137335132
264600000 9.05865018705409
264700000 3.60911103004

290600000 1.9049112836759374
290700000 7.604110534171954
290800000 7.758617061340087
290900000 7.869204941543507
291000000 7.771493941449615
291100000 7.70075831194789
291200000 7.79706312074479
291300000 7.757513057889644
291400000 7.929124040900842
291500000 8.099237470333042
291600000 2.8819430486345867
291700000 8.279432199086958
291800000 7.790059318272904
291900000 4.305146960838352
292000000 7.106743683251435
292100000 7.8821826165639015
292200000 7.775634943145834
292300000 7.846672149960808
292400000 7.784228341455664
292500000 7.631913004930577
292600000 7.817341296668333
292700000 7.941283886613867
292800000 7.672556246095009
292900000 4.972879624312588
293000000 4.846457780565304
293100000 7.587761514241537
293200000 7.891914886415179
293300000 7.792167614536021
293400000 7.876459130043712
293500000 8.07169583625586
293600000 8.001497852382606
293700000 7.853440121877685
293800000 7.950721151184374
293900000 7.403356686532975
294000000 3.6564383379706564
294100000 7.7245497

319500000 2.3308038306281733
319600000 2.2798055101669727
319700000 2.3372698001625687
319800000 2.4793340517923816
319900000 2.4832056187590914
320000000 2.628948379366277
320100000 2.426858607927644
320200000 2.608862744464606
320300000 2.3712455022301993
320400000 2.321626670415368
320500000 2.43895269545117
320600000 2.393688875226328
320700000 2.0220510994511462
320800000 2.3482092398047545
320900000 2.11392546407087
321000000 1.6946545831278188
321100000 1.2581636299713175
321200000 1.2261484434327907
321300000 1.2153534983348853
321400000 1.4368950523347972
321500000 2.352727360324616
321600000 2.6153286153126056
321700000 2.183881608190767
321800000 1.9532470008871163
321900000 2.361504822894085
322000000 2.3500143176172794
322100000 2.2834345036820167
322200000 2.375147314295505
322300000 2.599442183266121
322400000 2.3602933746449835
322500000 2.3447052244787177
322600000 2.3794918280859174
322700000 2.351156396895465
322800000 2.1813449755864944
322900000 2.0857008868931373


348100000 1.445257011152649
348200000 1.494739073359665
348300000 1.647859606123078
348400000 1.4051996348163007
348500000 1.3611709643548586
348600000 1.6179906725547095
348700000 1.8396080988152326
348800000 1.2727953821362317
348900000 1.2533377546064701
349000000 1.4774517556483528
349100000 1.4017393230575743
349200000 1.0510336515757588
349300000 0.6759207677494348
349400000 0.7221165870737103
349500000 0.7356200923520644
349600000 0.9136338556166193
349700000 1.1255178731247153
349800000 1.4790364858109486
349900000 1.202728516715894
350000000 1.2950857436076264
350100000 1.3553180459898517
350200000 1.253492324197346
350300000 1.3318703856984555
350400000 1.439124863309028
350500000 1.3470581360474825
350600000 1.4947974525836671
350700000 1.3625332937714272
350800000 1.3665233292675931
350900000 1.2913504260213946
351000000 1.2384129806861508
351100000 1.2330696527551412
351200000 1.3346899476554925
351300000 1.2367464044120762
351400000 1.2737591212507846
351500000 1.06095962

375900000 0.060962301340268726
376000000 0.058765823280505336
376100000 0.052167861909702753
376200000 0.06004745692973601
376300000 0.05921025851653217
376400000 0.05764264808453057
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.24533357516890367
376600000 0.30992478008337615
376700000 0.3492618563756883
376800000 0.3198073903718213
376900000 0.2978020549175223
377000000 0.294778380565136
377100000 0.27979959975215263
377200000 0.24646044046967625
377300000 0.24382286317352322
377400000 0.23844918478743635
377500000 0.2372368617081046
377600000 0.20927694564359517
377700000 0.2178220722389221
377800000 0.23271667216446795
377900000 0.22307378810775016
378000000 0.2085869247121937
378100000 0.1565969996976316
378200000 0.201693000280369
378300000 0.21430286954597236
378400000 0.2146549870888909
378500000 0.243174148005516
378600000 0.2554995180003597
378700000 0.24986895976754625
378800000 0.20508736516317658
378900000 0.04705367600151632
379000000 0.